In [1]:
## Standard libraries
import os
import numpy as np
import pandas as pd
import random
import math
import json
from functools import partial, reduce
import argparse
import copy
import time

from typing import TextIO, Callable, Collection, Dict, Iterator, List, Tuple, Type, TypeVar
T = TypeVar("T", bound="EyetrackingClassifier")

## PyTorch
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# PyTorch Lightning
import pytorch_lightning as pl

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "saved_models/transformer"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


Seed set to 42


Device: cuda:0


## Setup

In [2]:
# Global variables
NUM_FEATURES = 14
NUM_FIX = 30 
NUM_CLASSES = 31

In [3]:
# Helper functions

def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask

def mse_loss(target, input, mask):
    out = (input[mask]-target[mask])**2
    return out.mean()

def mask_with_tokens_3D(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    reduced = torch.any(mask, dim=-1, keepdim=True)
    expanded = reduced.expand_as(mask)
    return expanded

def get_mask_subset_with_prob_3D(mask, prob):
    batch, num_fix, num_features, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * num_fix)

    num_tokens = mask.sum(dim=-2, keepdim=True)
    mask_excess = (mask.cumsum(dim=-2)[:,:,0] > (num_tokens[:,:,0] * prob).ceil())
    mask_excess = mask_excess[:, :max_masked]

    rand = torch.rand((batch, num_fix, num_features), device=device).masked_fill(~mask, -1e9)
    _, sampled_indices = rand.topk(max_masked, dim=-2)
    sampled_indices = (sampled_indices[:,:,0] + 1).masked_fill_(mask_excess, 0)

    new_mask = torch.zeros((batch, num_fix + 1), device=device)
    new_mask.scatter_(-1, sampled_indices, 1)
    new_mask = new_mask[:, 1:].bool()
    
    return new_mask.unsqueeze_(2).expand(-1,-1, num_features)
    

def prob_mask_like_3D(t, prob):
    temp = torch.zeros_like(t[:,:,0]).float().uniform_(0, 1) < prob
    return temp.unsqueeze_(2).expand(-1,-1, NUM_FEATURES)
    
    
def pad_group_with_zeros(group, target_rows):
    # Calculate the number of rows to add
    num_missing_rows = target_rows - len(group)
    if num_missing_rows > 0:
        # Create a DataFrame with the required number of padding rows
        # input padding
        zero_rows = pd.DataFrame(0.3333, index=range(num_missing_rows), columns=group.columns)
        # Label padding
        # zero_rows.iloc[:, 0] = 31
        # Concatenate the group with the zero rows
        group = pd.concat([group, zero_rows], ignore_index=True)
    return group

# class ToTensor(object):
#     """Convert Series in sample to Tensors."""

#     def __call__(self, sample):
#         trial, label = sample['trial'], sample['label']
#         trial = torch.from_numpy(trial).float()
#         label = torch.from_numpy(label).float()
#         return trial, label

In [4]:
class CustomPositionalEncoding(nn.Module):
    """Learnable positional encoding for both features and fixations
    Assumes input `x` is of shape [batch_size, fixations, embed_dim]"""
    def __init__(self, fixations = 30, features = None):
        super(CustomPositionalEncoding, self).__init__()
        
        # Initialize a learnable positional encoding matrix
        self.encoding = nn.Parameter(torch.zeros(fixations, features)).to(device)
        nn.init.xavier_uniform_(self.encoding)  # Xavier initialization for better training stability
        
    def forward(self, x, mask = None):
        if mask is not None:
            # Apply the mask to ignore padded positions
            pos_encoding = self.encoding  * mask
        else:
            pos_encoding = self.encoding
        return x + pos_encoding

In [5]:
class AnnasPositionalEncoding(nn.Module):
    """Learnable positional encoding for both features and fixations
    Assumes input `x` is of shape [batch_size, fixations, embed_dim]"""
    def __init__(self, fixations = int, features = int):
        super(AnnasPositionalEncoding, self).__init__()
        
        # Initialize a learnable positional encoding matrix for fixations
        self.fix_encoding = nn.Parameter(torch.zeros(fixations, 1)).to(device)
        nn.init.xavier_uniform_(self.fix_encoding)  # Xavier initialization for better training stability
        self.fix_encoding = self.fix_encoding.expand(-1, features)
        
        # Initialize a learnable positional encoding matrix for features
        self.feat_encoding = nn.Parameter(torch.zeros(1, features)).to(device)
        nn.init.xavier_uniform_(self.feat_encoding)  # Xavier initialization for better training stability
        self.feat_encoding = self.feat_encoding.expand(fixations, -1)
        
        self.encoding = self.fix_encoding + self.feat_encoding
        
    def forward(self, x, mask = None):
        if mask is not None:
            # Apply the mask to ignore padded positions
            pos_encoding = self.encoding  * mask
        else:
            pos_encoding = self.encoding
        return x + pos_encoding

In [6]:
class AbsolutePositionalEmbedding(nn.Module):
    def __init__(self, dim, max_seq_len=278, l2norm_embed = False):
        super().__init__()
        self.scale = dim ** -0.5 if not l2norm_embed else 1.
        self.max_seq_len = max_seq_len
        self.l2norm_embed = l2norm_embed
        self.emb = nn.Embedding(max_seq_len, dim)

    def forward(self, x, pos = None, seq_start_pos = None, mask = None):
        seq_len, device = x.shape[1], x.device
        assert seq_len <= self.max_seq_len, f'you are passing in a sequence length of {seq_len} but your absolute positional embedding has a max sequence length of {self.max_seq_len}'

        #if not exists(pos):
        pos = torch.arange(seq_len, device = device)

        #if exists(seq_start_pos):
        #    pos = (pos - seq_start_pos[..., None]).clamp(min = 0)

        pos_emb = self.emb(pos)
        pos_emb = pos_emb * self.scale
        
        if mask is not None:
            # Apply the mask to ignore padded positions
            pos_emb = pos_emb * mask
            
        return l2norm(pos_emb) if self.l2norm_embed else pos_emb


In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=278):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model%2 != 0:
            pe[:, 1::2] = torch.cos(position * div_term)[:,0:-1]
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).to(device)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x, mask = None):
        pos_enc = self.pe[:, :x.size(1)]
        if not(mask == None):
            mask = mask.to(device)
            pos_enc = pos_enc*mask
        return x + pos_enc

## Data preparation

In [8]:
#LinguisticFeature = Callable[[Tuple[str]], Tuple[torch.Tensor]]

# def apply_standardization(x, m, sd):
#     nonzero_sd = sd.clone()
#     nonzero_sd[sd == 0] = 1
#     x = torch.from_numpy(x).float()
#     res = (x - m.unsqueeze(0)) / nonzero_sd.unsqueeze(0)
#     return res

def apply_standardization(x, m, sd):
    nonzero_sd = sd.clone()
    nonzero_sd[sd == 0] = 1
    x = torch.from_numpy(x).float()
    x_zeros = x[x.sum(dim=(1)) == 0]
    x_zeros[x_zeros==0] = -5
    x_non_zeros = x[x.sum(dim=(1)) != 0]
    x_non_zeros = (x_non_zeros - m.unsqueeze(0)) / nonzero_sd.unsqueeze(0)
    res = torch.cat((x_non_zeros, x_zeros), axis =0)
    return res


def aggregate_per_subject(subjs, y_preds, y_preds_class, y_trues):
    y_preds = np.array(y_preds)
    y_preds_class = np.array(y_preds_class)
    y_trues = np.array(y_trues)
    subjs = np.array(subjs).flatten()
    y_preds_subj = []
    y_preds_class_subj = []
    y_trues_subj = []
    subjs_subj = np.unique(subjs)
    for subj in subjs_subj:
        subj = subj.item()
        y_pred_class_subj = y_preds_class[subjs == subj]
        y_pred_subj = y_preds[subjs == subj]
        y_true_subj = y_trues[subjs == subj]
        assert len(np.unique(y_true_subj)) == 1, f"No unique label: subj={subj}"
        y_trues_subj.append(np.unique(y_true_subj).item())
        y_preds_subj.append(np.mean(y_pred_subj).item())
        if sum(y_pred_class_subj) >= (len(y_pred_class_subj) / 2):
            y_preds_class_subj.append(1)
        else:
            y_preds_class_subj.append(0)
    return subjs_subj, y_preds_subj, y_preds_class_subj, y_trues_subj

def getmeansd(dataset, batch: bool = False):  # removing rows of 0s
    if batch:
        # Anna added preprocessing from ndarray to torch
        tensors = [X for X, _, _, _ in dataset]  #torch.from_numpy(X).float()
        tensors = torch.cat(tensors, axis=0)
        # remove padded tensors
        tensors = tensors[tensors.sum(dim=(1,2)) != 0]   #tensors[tensors.sum(dim=(1, 2)) != 0]
        # remove rows of 0s from the computation
        sentences, timesteps, features = tensors.size()
        subset = tensors.sum(dim=(2)) != 0
        subset = subset.view(sentences, timesteps, 1)
        subset = subset.expand(sentences, timesteps, features)
        result = tensors[subset].view(-1, features) 
        
        means = torch.mean(result, dim=(0))
        sd = torch.std(result, dim=(0))
        return means, sd
    else:
        tensors = [torch.from_numpy(X).float() for X, _, _, _ in dataset] # Anna added , was [X for X, _, _ in dataset]
        tensors = torch.cat(tensors, axis=0)
        # remove padded tensors
        tensors = tensors[tensors.sum(dim=1) != 0]
        means = torch.mean(tensors, 0)
        sd = torch.std(tensors, 0)
        return means, sd
    
    
# def get_params(paramdict) -> dict:
#     selected_pars = dict()
#     for k in paramdict:
#         selected_pars[k] = random.sample(list(paramdict[k]), 1)[0]
#     return selected_pars

In [9]:
def aggregate_speed_per_subject(subjs, y_preds, y_trues):
    y_preds = np.array(y_preds)
    #y_preds_class = np.array(y_preds_class)
    y_trues = np.array(y_trues)
    subjs = np.array(subjs).flatten()
    y_preds_subj = []
    y_trues_subj = []
    subjs_subj = np.unique(subjs)
    for subj in subjs_subj:
        subj = subj.item()
        y_pred_subj = y_preds[subjs == subj]
        y_true_subj = y_trues[subjs == subj]
        assert len(np.unique(y_true_subj)) == 1, f"No unique label: subj={subj}"
        y_trues_subj.append(np.unique(y_true_subj).item())
        y_preds_subj.append(np.mean(y_pred_subj).item())

    return subjs_subj, y_preds_subj, y_trues_subj

In [10]:
class EyetrackingDataPreprocessor(Dataset):
    """Dataset with the long-format sequence of fixations made during reading by dyslexic 
    and normally-developing Russian-speaking monolingual children."""

    def __init__(
        self, 
        csv_file, 
        transform=None, 
        target_transform=None,  
        num_folds: float = 10,
        ):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            target_transform (callable, optional): Optional transform to be applied
                on a label.
        """
        data = pd.read_csv(csv_file)
        
        # changing dyslexia labels to 0 and 1
        if {'group'}.issubset(data.columns):   # not the case for pretrain dataset
            data['group'] = data['group'] + 0.5
        
        # log-transforming frequency
        to_transform = ['frequency', 'predictability', 'fix_dur'] #
        for column in to_transform:
            data[column] = data[column].apply(lambda x: np.log(x) if x > 0 else 0) 
        
        # drop columns we don't use
        data = data.drop(columns = ['fix_x', 'fix_y', 'fix_index'])  
        
        # center reading sopeed in case we need to predict it
        if {'Reading_speed'}.issubset(data.columns):
            data['Reading_speed'] = (data['Reading_speed'] - data['Reading_speed'].mean())/data['Reading_speed'].std(ddof=0)
        
        if {'sex', 'Grade'}.issubset(data.columns):
            data = data.drop(columns = ['sex', 'Grade'])
            
        convert_columns = ['direction']
        
        if {'IQ', 'Sound_detection', 'Sound_change'}.issubset(data.columns):
            data = data.drop(columns = ['IQ', 'Sound_detection', 'Sound_change'])
        
        for column in convert_columns:
            prefix = column + '_dummy'
            data = pd.concat([data, pd.get_dummies(data[column], 
                                    prefix=prefix)], axis=1)
            data = data.drop(columns = column)

        data.dropna(axis = 0, how = 'any', inplace = True)

            
        # rearrange columns (I need demogrpahic information to come last)
#         cols = ['item', 'subj', 'group', 'Reading_speed', 'fix_dur', 'landing', 'word_length',
#                  'predictability', 'frequency', 'number.morphemes', 'next_fix_dist',
#                  'sac_ampl', 'sac_angle', 'sac_vel', 'rel.position', 'direction_dummy_DOWN',
#                  'direction_dummy_LEFT', 'direction_dummy_RIGHT', 'direction_dummy_UP',
#                  'sex', 'Age', 'Grade_dummy_1', 'Grade_dummy_2', 'Grade_dummy_3', 'Grade_dummy_4',
#                  'Grade_dummy_5', 'Grade_dummy_6']
        if {'Reading_speed'}.issubset(data.columns):
            cols = ['item', 'subj', 'group', 'Reading_speed', 'fix_dur',
                   'landing', 'word_length', 'predictability', 'frequency', 
                    'number.morphemes', 'next_fix_dist', 'sac_ampl', 'sac_angle', 
                    'sac_vel', 'rel.position', 'direction_dummy_LEFT', 
                    'direction_dummy_RIGHT', 'direction_dummy_DOWN'] # temporary
        else:
            cols = ['item', 'subj', 'fix_dur',
                   'landing', 'word_length', 'predictability', 'frequency', 
                    'number.morphemes', 'next_fix_dist', 'sac_ampl', 'sac_angle', 
                    'sac_vel', 'rel.position', 'direction_dummy_LEFT', 
                    'direction_dummy_RIGHT', 'direction_dummy_DOWN'] # temporary
        data = data[cols]
        
        # Record features that are used for prediction
        if {'Reading_speed'}.issubset(data.columns):
            self._features = [i for i in data.columns if i not in ['group', 'item', 'subj', 'Reading_speed']]
        else:
            self._features = [i for i in data.columns if i not in ['item', 'subj']]
        self._data = pd.DataFrame()
        # Add sentence IDs and subject IDs
        self._data["sn"] = data["item"]
        self._data["subj"] = data["subj"]
        # Add labels
        if {'Reading_speed'}.issubset(data.columns):
            self._data["group"] = data["group"]
            self._data["reading_speed"] = data["Reading_speed"]
        else:
            self._data["group"] = -1
            self._data["reading_speed"] = -1
        
        # Add features used for prediction
        for feature in self._features:
            self._data[feature] = data[feature]

#       # Distribute subjects across stratified folds
        self._num_folds = num_folds
        self._folds = [[] for _ in range(num_folds)]
        just_subjects = self._data["subj"].unique()
        random.shuffle(just_subjects)
        for i, subj in enumerate(just_subjects):
            self._folds[i % num_folds].append(subj)
#         dyslexic_subjects = self._data[self._data["group"] == 1]["subj"].unique()
#         control_subjects = self._data[self._data["group"] == 0]["subj"].unique()
#         random.shuffle(dyslexic_subjects)
#         random.shuffle(control_subjects)
#         for i, subj in enumerate(dyslexic_subjects):
#             self._folds[i % num_folds].append(subj)
#         for i, subj in enumerate(control_subjects):
#             self._folds[num_folds - 1 - i % num_folds].append(subj)
        for fold in self._folds:
            random.shuffle(fold)

    def _iter_trials(self, folds: Collection[int]) -> Iterator[pd.DataFrame]:
        # Iterate over all folds
        for fold in folds:
            # Iterate over all subjects in the fold
            for subj in self._folds[fold]:       # Anna: subj in fold?
                subj_data = self._data[self._data["subj"] == subj]
                # Iterate over all sentences this subject read
                for sn in subj_data["sn"].unique():
                    trial_data = subj_data[subj_data["sn"] == sn]
                    yield trial_data
                    
                    
    def iter_folds(
        self, folds: Collection[int]) -> Iterator[Tuple[torch.Tensor, torch.Tensor, int]]:
        for trial_data in self._iter_trials(folds):
            predictors = trial_data[self._features].to_numpy()
            #predictors = np.reshape(predictors, (int(len(predictors)/278), 278, predictors.shape[1]))
            label = trial_data["group"].unique().item()
            subj = trial_data["subj"].unique().item()
            reading_speed = trial_data["reading_speed"].unique().item()
            #  X = (time_steps, features)
            X = predictors
            y = torch.tensor(label, dtype=torch.float)
            rs = torch.tensor(reading_speed , dtype=torch.float)
            yield X, y, subj, rs
                    

    @property
    def num_features(self) -> int:
        """Number of features per word (excluding word vector dimensions)."""
        return len(self._features)
    

    @property
    def max_number_of_sentences(self):
        data_copy = self._data.copy()
        max_s_count = data_copy.groupby(by="subj").sn.unique()
        return max([len(x) for x in max_s_count])

In [11]:
class EyetrackingDataset(Dataset):
    def __init__(
        self,
        preprocessor: EyetrackingDataPreprocessor,
       # word_vector_model: WordVectorModel,
        folds: Collection[int],
        batch_subjects: bool = False,
    ):
        self.sentences = list(preprocessor.iter_folds(folds))
        self._subjects = list(np.unique([subj for _, _, subj, _ in self.sentences]))
        self.num_features = preprocessor.num_features# + word_vector_model.dimensions()
        self.batch_subjects = batch_subjects
        #self.max_sentence_length = preprocessor.max_sentence_length
        self.max_number_of_sentences = preprocessor.max_number_of_sentences

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor, int]:
        if self.batch_subjects:
            subject = self._subjects[index]
            subject_sentences = [
                (X, y, subj, rs) for X, y, subj, rs in self.sentences if subj == subject
            ]
            X = torch.stack([torch.FloatTensor(X) for X, _, _, _ in subject_sentences]) #[X for X, _, _ in subject_sentences] #torch.FloatTensor([X for X, _, _ in subject_sentences])
            y = torch.stack([y for _, y, _, _ in subject_sentences]).unique().squeeze() 
            rs = torch.stack([rs for _, _, _, rs in subject_sentences]).unique().squeeze()
            return X, y, subject, rs

        else:
            X, y, subj, rs = self.sentences[index] 
            return X, y, subj, rs

    def __len__(self) -> int:
        if self.batch_subjects:
            return len(self._subjects)
        else:
            return len(self.sentences)

    def standardize(self, mean: torch.Tensor, sd: torch.Tensor):
        self.sentences = [
            (apply_standardization(X, mean, sd), y, subj, rs)
            for X, y, subj, rs in self.sentences
        ]

## Model

In [12]:
class EncoderLayer(nn.Module):  
    def __init__(self,
                dim_upscale = int,
                inner_dim_upscale = int,
                num_heads = int, 
                num_layers = int, 
                dropout = 0
                ):

        super().__init__()
        self.num_heads = num_heads
        self.dropout = dropout
        self.dim_upscale = dim_upscale
        self.inner_dim_upscale = inner_dim_upscale
        
        # layer norm for multi-head attention
        self.attn_layer_norm = nn.LayerNorm(self.dim_upscale)
        # layer norm for feedforward network
        self.ffn_layer_norm = nn.LayerNorm(self.dim_upscale)
        
        self.attention = nn.MultiheadAttention(embed_dim = self.dim_upscale,  
                                                 num_heads = self.num_heads, 
                                                 bias = True,
                                                 batch_first = True)
        # feed forward
        self.ff = nn.Sequential(
            nn.Linear(self.dim_upscale, self.inner_dim_upscale, bias = True),
            nn.LayerNorm(self.inner_dim_upscale),
            nn.ReLU(inplace=True),
            nn.Dropout(self.dropout),
            nn.Linear(self.inner_dim_upscale, self.dim_upscale, bias = True)
        )
        

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor):
        # pass embeddings through multi-head attention
        x, attn_probs = self.attention(src, src, src, src_mask)

        # residual add and norm
        first_out = self.attn_layer_norm(x + src)

        # position-wise feed-forward network
        x2 = self.ff(first_out)

        # residual add and norm
        second_out = self.ffn_layer_norm(x2 + first_out)

        return second_out, attn_probs


In [13]:
class Encoder(nn.Module):
    def __init__(self, 
                dim_upscale = int,
                inner_dim_upscale = int,
                num_heads = int, 
                num_layers = int, 
                dropout = 0):

        super().__init__()
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.dropout = dropout
        self.dim_upscale = dim_upscale
        self.inner_dim_upscale = inner_dim_upscale
        
        # create n_layers encoders 
        self.layers = nn.ModuleList([EncoderLayer(
                                    dim_upscale = self.dim_upscale,
                                    num_heads = self.num_heads, 
                                    inner_dim_upscale = self.inner_dim_upscale,
                                    dropout = self.dropout)
                                     for layer in range(self.num_layers)])


    def forward(self, src: torch.Tensor, src_mask: torch.Tensor):

        # pass the sequences through each encoder
        for layer in self.layers:
            src, attn_probs = layer(src, src_mask)

        self.attn_probs = attn_probs

        return src

In [14]:
class EyetrackingClassifier(nn.Module):
    def __init__(self, input_size: int, config):
        super().__init__()
        self.initialize_model(input_size, config)
        self.config = config

    def initialize_model(self, input_size: int, config):
        raise NotImplementedError()

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError()

    def _predict(self, X: torch.Tensor, identity: bool = False, subj_mean: bool = False, pretrain: bool = False) -> torch.Tensor:
        if pretrain:
            y_preds, labels, mask = self(X, identity, pretrain=True)
            return y_preds, labels, mask
        else:
            y_preds, labels, mask = self(X, identity, pretrain)
            return y_preds, labels, mask
        
    @classmethod
    def train_model(
        cls: Type[T],
        data: EyetrackingDataset,
        min_epochs: int = 15,
        max_epochs: int = 300,
        dev_data: EyetrackingDataset = None,
        device: str = "cuda",
        config = None,
        patience = 20,
        pretrained_model: T = None,
        **kwargs,
    ) -> Tuple[T, int]:
        model = pretrained_model or cls(data.num_features, config, **kwargs) 
        model.to(device)
        model.train()
        optimizer = Adam(model.parameters(), lr=config["lr"])
        epoch_count = 0
        best_losses = [float("inf")] * patience
        for epoch in range(max_epochs):
            # reshuffle data in each epoch
            loader = torch.utils.data.DataLoader(
                data,
                batch_size=BATCH_SIZE,
                shuffle=True,
            )
            epoch_count += 1
            epoch_loss = 0
            for X, _, _, _ in loader:
                X = X.to(device)
                train_preds, labels, mask = model._predict(X, identity = False) 
                loss = mse_loss(
                    labels,
                    train_preds,
                    mask
                )
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            #avg_loss = epoch_loss/math.ceil(len(loader.dataset)/BATCH_SIZE)  # TODO: delete later
            #print(f"Epoch {epoch} done. Loss: {avg_loss}")
            
            if dev_data is not None:
                dev_loss = model.evaluate(dev_data, metric="loss", device=device)
                model.train()
                # print(f"Dev loss: {dev_accuracy} in Epoch {epoch}")
                if epoch > min_epochs and all(dev_loss > i for i in best_losses):
                    epoch_count -= patience - best_losses.index(min(best_losses))
                    break
                else:
                    best_losses.pop(0)
                    best_losses.append(dev_loss)
        return model

    def evaluate(
        self,
        data: EyetrackingDataset,
        metric: str = "loss",
        device: str = "cuda",
    ) -> Tuple[float, float, float, float]:
        self.to(device)
        self.eval()
        loader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE)
        loss = 0
        for X, _, _, _ in loader:
            X = X.to(device)
            dev_preds, labels, mask = self._predict(X, identity = False)
            dloss = mse_loss(
                labels,
                dev_preds,
                mask
            )
            loss += dloss.item() 
        avg_loss = loss /math.ceil(len(loader.dataset)/BATCH_SIZE) 
        if metric == "loss":
            return avg_loss
        else:
            raise ValueError(f"Unknown metric '{metric}'")


In [15]:
class TransformerClassifier(EyetrackingClassifier):
    def initialize_model(
        self, 
        input_size: int, 
        config,
        embed_dim = NUM_FIX,
        d_model = NUM_FEATURES,
        dim_upscale = 128,
        inner_dim_upscale = 4*128,
        num_heads = 1, 
        num_layers = 1, 
        dropout = 0.15,
        mask_prob = 0.2,
        pad_token_id = -5,
        mask_ignore_token_ids = []
        ):
        
        self.embed_dim = embed_dim
        self.d_model = d_model
        self.mask_prob = config["mask_prob"]
        self.num_heads = config["num_heads"]
        self.num_layers = config["num_layers"]
        self.dropout = config["dropout"]
        self.dim_upscale = config["upscale_dim"]
        self.inner_dim_upscale = config["inner_dim"]

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, self.pad_token_id])
        
        self.positional_encoding = AnnasPositionalEncoding(fixations = self.embed_dim, 
                                                           features = self.d_model)
        
        self.upscale = nn.Linear(self.d_model, self.dim_upscale, bias = True)
        self.downscale = nn.Linear(self.dim_upscale, self.d_model, bias = True)
        
        self.encoder = Encoder(dim_upscale = self.dim_upscale, 
                               num_heads = self.num_heads, 
                               num_layers = self.num_layers,
                               inner_dim_upscale = self.inner_dim_upscale, 
                               dropout = self.dropout)

    def forward(self, input: torch.Tensor, identity = False, pretrain: bool = False):
        
        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens_3D(input, self.mask_ignore_token_ids) 
        mask = get_mask_subset_with_prob_3D(~no_mask, self.mask_prob)
        hidden = no_mask + mask # all elements that the model will not attend to

        masked_seq = input.clone().detach().to(device)
        
        #  positional encoding
        masked_seq_pos = self.positional_encoding(masked_seq, mask = None) #for hidden fixations ~ no_mask

        # derive labels to predict
        labels = input.masked_fill(~mask, self.pad_token_id)
    
        if identity:
            attn_mask = no_mask[:,:,0]
        else:
            attn_mask = hidden[:,:,0]
        
        # Upscaling
        masked_seq_upscaled = self.upscale(masked_seq_pos)
        
        # Encoder
        out = self.encoder(masked_seq_upscaled, 
                           attn_mask)  
        out = self.downscale(out)

        return out, labels, mask


## Training

In [16]:
hyperparameter_space = {
    "transformer": {
        "lr": np.linspace(1e-03, 1e-05, num=15), 
        "num_layers": [1, 2, 4, 8, 16],      
        "dropout": np.linspace(0, 0.5, num=15),
        "mask_prob": np.linspace(0, 0.5, num=15),
        "upscale_dim": [14, 32, 64, 128, 256, 512, 1024],
        # "num_heads" is specified in get_params_nested()
        # "inner_dim" is specified in get_params_nested()
    }
}

nested_space = {
    "14": [1, 2, 7, 14],
	"32": [1, 4, 8, 16, 32],
	"64": [1, 8, 16, 32, 64],
	"128": [1, 8, 32, 64, 128],
	"256": [1, 32, 64, 128, 256],
	"512": [32, 64, 128, 256, 512],
	"1024": [32, 64, 128, 256, 512], 
}


def get_params_nested(paramdict, nested) -> dict:
    selected_pars = dict()
    for k in paramdict:
        selected_pars[k] = random.sample(list(paramdict[k]), 1)[0] 
    factor = [1, 2, 4, 8]    
    if selected_pars["upscale_dim"] == 14:
        selected_pars["num_heads"] = random.sample(list(nested["14"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([14*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 32:
        selected_pars["num_heads"] = random.sample(list(nested["32"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([32*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 64:
        selected_pars["num_heads"] = random.sample(list(nested["64"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([64*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 128:
        selected_pars["num_heads"] = random.sample(list(nested["128"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([128*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 256:
        selected_pars["num_heads"] = random.sample(list(nested["256"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([256*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 512:
        selected_pars["num_heads"] = random.sample(list(nested["512"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([512*x for x in factor], 1)[0]
    if selected_pars["upscale_dim"] == 1024:
        selected_pars["num_heads"] = random.sample(list(nested["1024"]), 1)[0]
        selected_pars["inner_dim"] = random.sample([1024*x for x in factor], 1)[0]
        
    return selected_pars

In [17]:
# default: --no-tune --wordvectors none --model cnn --subjpred
parser = argparse.ArgumentParser(description="Run Russian Eye-Movement Pretraining")
parser.add_argument("--model", dest="model")
parser.add_argument("--tunesets", type=int, default=1000)
parser.add_argument("--tune", dest="tune", action="store_true")
parser.add_argument("--no-tune", dest="tune", action="store_false")
parser.add_argument("--pretrain", dest="pretrain", action="store_true")
parser.add_argument("--seed", dest="seed", type=int, default=42)
parser.add_argument("--cuda", dest="cudaid", default=0)
parser.set_defaults(tune=True) 
parser.set_defaults(model = "transformer")
args = parser.parse_args(args=[]) # modified to work with jupyter notebook

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

start_time = time.time()

if args.model == "transformer":
    MODEL_CLASS = TransformerClassifier 
    
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    device = torch.device(f'cuda:{args.cudaid}')
    
NUM_FOLDS = 10
NUM_TUNE_SETS = args.tunesets
tune = args.tune

BATCH_SIZE = 128
BATCH_SUBJECTS = False

file = "data/30fixations_RSC_and_children.csv" # combined: children (293) and adults (114), N = 407


NUM_FOLDS = 4
NUM_TUNE_SETS = 2

if tune:
    used_test_params = []
    parameter_sample = [
        get_params_nested(hyperparameter_space["transformer"], nested_space) for _ in range(NUM_TUNE_SETS)
    ]

tprs_folds = {}
    

# load and preprocess data for training
preprocessor = EyetrackingDataPreprocessor(
    csv_file = file, 
   num_folds = NUM_FOLDS
)

test_accuracies = []

dev_fold = 2   # dev fold stays the same
parameter_evaluations = np.zeros(shape=(NUM_FOLDS, NUM_TUNE_SETS))
if tune:
    train_folds = [
        fold
        for fold in range(NUM_FOLDS)
        if fold != dev_fold
    ]
    train_dataset = EyetrackingDataset(
        preprocessor,
        folds=train_folds,
        batch_subjects=BATCH_SUBJECTS,
    )
    mean, sd = getmeansd(train_dataset, batch=BATCH_SUBJECTS)
    train_dataset.standardize(mean, sd)
    dev_dataset = EyetrackingDataset(
        preprocessor,
        folds=[dev_fold],
        batch_subjects=BATCH_SUBJECTS,
    )
    dev_dataset.standardize(mean, sd)
    for tune_set in range(NUM_TUNE_SETS):
        running_model = copy.deepcopy(MODEL_CLASS)
        if tune_set%100 == 0:
            print(f'tune set {tune_set}')
        if args.pretrain:
            pretrained_model = next(pretrained_models)
        else:
            pretrained_model = None
        model = None
        model = running_model.train_model(
            train_dataset,
            min_epochs=15,
            max_epochs=300,
            dev_data=dev_dataset,
            pretrained_model=pretrained_model,
            device=device,
            config=parameter_sample[tune_set],
        )
        tune_accuracy = model.evaluate(
            data=dev_dataset,
            device=device,
            metric="loss"
        )
        parameter_evaluations[dev_fold, tune_set] = tune_accuracy
    # Select best parameter set
    mean_dev_loss = np.mean(parameter_evaluations, axis=0)
    best_parameter_set = np.argmin(mean_dev_loss)
    params_test = parameter_sample[best_parameter_set]
    print(f'best performing parameter for dev fold ', dev_fold, ": ", params_test)
    used_test_params.append(params_test)
    best_pretrained_model = None
    
# Evaluation    
for test_fold in range(NUM_FOLDS):
    if dev_fold == test_fold:
        continue
    print(f'test fold {test_fold}')
    running_model = copy.deepcopy(MODEL_CLASS)

    train_folds = [
        fold for fold in range(NUM_FOLDS) if fold != test_fold and fold != dev_fold
    ]

    train_dataset = EyetrackingDataset(
        preprocessor,
        folds=train_folds,
        batch_subjects=BATCH_SUBJECTS,
    )
    mean, sd = getmeansd(train_dataset, batch=BATCH_SUBJECTS)
    train_dataset.standardize(mean, sd)
    dev_dataset = EyetrackingDataset(
        preprocessor,
        folds=[dev_fold],
        batch_subjects=BATCH_SUBJECTS
    )
    dev_dataset.standardize(mean, sd)
    test_dataset = EyetrackingDataset(
        preprocessor,
        folds=[test_fold],
        batch_subjects=BATCH_SUBJECTS,
    )
    test_dataset.standardize(mean, sd)
    model = running_model.train_model(
        train_dataset,
        min_epochs=15,
        max_epochs=300,
        dev_data=dev_dataset,
        pretrained_model=best_pretrained_model,
        device=device,
        config=params_test,
    )
    #print(f'test accuraccy fold ', test_fold)
    test_loss = model.evaluate(
        test_dataset,
        device=device,
        metric="loss",
    )
    print("test loss fold", test_fold, ":", test_loss)
    test_accuracies.append(test_loss)

if tune:
    print("used test params: ", used_test_params)
print(
    "mean:",
    np.mean(test_accuracies, axis=0),
    "std:",
    np.std(test_accuracies, axis=0) / np.sqrt(NUM_FOLDS),
)

final_scores_mean = np.mean(test_accuracies, axis=0)
final_scores_std = np.std(test_accuracies, axis=0) / np.sqrt(NUM_FOLDS-1)
time_elapsed = time.time() - start_time
out_str = ""
with open(f"results.txt", "w") as f:
    out_str += f"${round(final_scores_mean,6):1.6f}\db{{{round(final_scores_std,6):1.6f}}}$"
    out_str += "\n"
    out_str += f"Time: {time_elapsed:7.2f} seconds.\n" 
    out_str += f"Test losses: {test_accuracies}\n"
    out_str += f"Used test params: {used_test_params}"
    f.write(out_str)


tune set 0
best performing parameter for dev fold  2 :  {'lr': 0.0002928571428571429, 'num_layers': 1, 'dropout': 0.0, 'mask_prob': 0.39285714285714285, 'upscale_dim': 64, 'num_heads': 8, 'inner_dim': 128}
test fold 0
test loss fold 0 : 0.00014099742807245448
test fold 1
test loss fold 1 : 0.00014598591301461403
test fold 3
test loss fold 3 : 0.00023352457302702
used test params:  [{'lr': 0.0002928571428571429, 'num_layers': 1, 'dropout': 0.0, 'mask_prob': 0.39285714285714285, 'upscale_dim': 64, 'num_heads': 8, 'inner_dim': 128}]
mean: 0.0001735026380380295 std: 2.124537500795365e-05


In [22]:
old_folds = preprocessor._folds


In [28]:
preprocessor._folds == old_folds

True